In [1]:
import sys
import dlib
import cv2
import os
import glob
import numpy as np
import time
import math

t1 = time.time()
current_path = os.getcwd()  # 获取当前路径
predictor_path = current_path + "\\model\\shape_predictor_68_face_landmarks.dat"
face_rec_model_path = current_path + "\\model\\dlib_face_recognition_resnet_model_v1.dat"
faces_folder_path = current_path + "\\faces\\"
people_path = current_path + "\\people\\"
data_path = current_path + "\\data\\"
detector = dlib.get_frontal_face_detector()
shape_predictor = dlib.shape_predictor(predictor_path)
face_rec_model = dlib.face_recognition_model_v1(face_rec_model_path)

vector_list=[]
path_name = []
for img_path in glob.glob(os.path.join(faces_folder_path, "*.jpg")):
    path0 = img_path.split("\\")[-1]
    path_name.append(path0)
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    b, g, r = cv2.split(img)
    img2 = cv2.merge([r, g, b])
    dets = detector(img, 1)   # 人脸标定
    for index, face in enumerate(dets):
        shape = shape_predictor(img2, face)   # 提取68个特征点
        face_descriptor = face_rec_model.compute_face_descriptor(img2, shape)   # 计算人脸的128维的向量
        path0 = str(path0).split(".")
        filePath = data_path + path0[0] + '.npy'
        vectors = np.array([])
        for i, num in enumerate(face_descriptor):
            vectors = np.append(vectors, num)
        np.save(filePath, vectors) 
t2 = time.time()-t1
print(str(t2)+"s"+"over！")

44.601675033569336sover！


In [2]:
import sys
import dlib
import cv2
import os
import glob
import numpy as np
import time
import math

def judge_people(data1,data2):
    chart = 0.55
    diff = 0
    for i in range(len(data1)):
        diff += (data1[i] - data2[i])**2#欧式距离
        #print(diff)
        if diff > chart  or diff < -chart:
            return False
    diff = np.sqrt(diff)
    if(diff < chart):
        #print(diff)
        return True
    else:
        return False 
    
t1 = time.time()
current_path = os.getcwd()  # 获取当前路径
predictor_path = current_path + "\\model\\shape_predictor_68_face_landmarks.dat"
face_rec_model_path = current_path + "\\model\\dlib_face_recognition_resnet_model_v1.dat"
faces_folder_path = current_path + "\\faces\\"
people_path = current_path + "\\people\\"
data_path = current_path + "\\data\\"

detector = dlib.get_frontal_face_detector()
shape_predictor = dlib.shape_predictor(predictor_path)
face_rec_model = dlib.face_recognition_model_v1(face_rec_model_path)

vector_list=[]
path_name = []
for img_path in glob.glob(os.path.join(data_path, "*.npy")):
    vectors = np.load(img_path)
    vector_list.append(vectors)
    path0 = img_path.replace(".npy",".jpg")
    path0 = path0.split("\\")[-1]
    path_name.append(path0)
t2 = time.time()-t1
print("载入用时"+str(t2)+"s")
since = time.time()
f = open('result.txt','w')

for img_path in glob.glob(os.path.join(people_path, "*.jpg")):
    path1 = img_path.split("\\")[-1]
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    b, g, r = cv2.split(img)
    img2 = cv2.merge([r, g, b])
    dets = detector(img, 1)   # 人脸标定
    for index, face in enumerate(dets):
        shape = shape_predictor(img2, face)   # 提取68个特征点
        face_descriptor = face_rec_model.compute_face_descriptor(img2, shape)   # 计算人脸的128维的向量
        vectors = np.array([])
        for i, num in enumerate(face_descriptor):
            vectors = np.append(vectors, num)
        number = 0
        #print(path1+":"+str(np.mean(vectors)))
        find_flag = 0
        for example in vector_list:
            if find_flag != 1:
                if True == judge_people(example,vectors):
                    print(path1+";"+path_name[number])
                    f.writelines(path1+";"+path_name[number]+"\n") 
                    find_flag = 1 
            number +=1
        if find_flag == 0:
            print(path1+";Null")
            f.writelines(path1+";Null"+"\n") 
        time_elapsed = time.time() - since
        f_str = (path1+"用时:"+str(time_elapsed)+"s"+"\n")
        print(f_str)
        since = time.time()
t2 = time.time()-t1
f.close()
print("总用时"+str(t2)+"s")

载入用时2.4827795028686523s
Parris_Glendening_0002.jpg;Parris_Glendening_0001.jpg
Parris_Glendening_0002.jpg用时:0.7335379123687744s

Pascal_Quignard_0003.jpg;Pascal_Quignard_0001.jpg
Pascal_Quignard_0003.jpg用时:1.0617327690124512s

Patrice_Chereau_0002.jpg;Patrice_Chereau_0001.jpg
Patrice_Chereau_0002.jpg用时:0.6674761772155762s

Paul_Greengrass_0001.jpg;Null
Paul_Greengrass_0001.jpg用时:0.6494593620300293s

Paul_Kariya_0001.jpg;Null
Paul_Kariya_0001.jpg用时:0.6524796485900879s

Peter_Greenaway_0002.jpg;Peter_Greenaway_0001.jpg
Peter_Greenaway_0002.jpg用时:0.684485673904419s

Phil_Gramm_0002.jpg;Phil_Gramm_0001.jpg
Phil_Gramm_0002.jpg用时:1.0487229824066162s

Princess_Caroline_0004.jpg;Princess_Caroline_0001.jpg
Princess_Caroline_0004.jpg用时:0.6494603157043457s

Princess_Elisabeth_0001.jpg;Princess_Elisabeth_0002.jpg
Princess_Elisabeth_0001.jpg用时:0.672478437423706s

Queen_Beatrix_0002.jpg;Queen_Beatrix_0001.jpg
Queen_Beatrix_0002.jpg用时:0.6634705066680908s

总用时9.968065023422241s
